In [2]:
import pandas as pd
import os
import shutil
import sys
import numpy
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, roc_curve

In [3]:
import time
t = time.time()

In [4]:
infile = "/repos/tables/glom_xml_split.tab"
df = pd.read_table(infile, usecols=["file_id", "split"])
# df['png'] = df.file_id.map(lambda x: x+".png")
df = df.set_index("file_id")["split"]
df.shape

(83,)

In [5]:
# df.set_index('id', inplace=True)


In [6]:
# indir = "/repos/data/glom/data_1024/glom_split/all"
# indir = "/repos/data/glom/data_512_subsample_2x/glom_split/all"
indir = "/repos/data/glom/data_256_subsample_4x/glom_split/all"
indir = "/repos/data/glom/data_128_subsample_8x/glom_split/all"
outdir = os.path.dirname(indir.rstrip('/'))
def datagen(indir):
    for dd in os.scandir(indir):
        for ff in os.scandir(dd.path):
#             if os.path.isdir(ff.path) or not (ff.name.endswith("png") or ff.name.endswith("json")):
            if os.path.isdir(ff.path) or not (ff.name.endswith("json")):
                continue
#             print(ff.name.split('-')[0], ff.path)
            yield (ff.name.split('-')[0], ff)
    
def gen_set(indir, outdir, df):
    for slideid, ff in datagen(indir):
        posnegset = os.path.basename(os.path.dirname(ff.path))
        set_ = df.loc[slideid]
        yield ff.path, os.path.join(outdir, set_, posnegset, ff.name)

In [7]:
gen = gen_set(indir, outdir, df)
sum((1 for _ in gen))

11710

In [8]:
posnegset = os.listdir(indir)
for _, set_ in df.drop_duplicates().items():
    for pn in posnegset:
        setdir = os.path.join(outdir, set_, pn)
        print(setdir)
        os.makedirs(setdir, exist_ok=True)

/repos/data/glom/data_128_subsample_8x/glom_split/train/normal
/repos/data/glom/data_128_subsample_8x/glom_split/train/glom
/repos/data/glom/data_128_subsample_8x/glom_split/test/normal
/repos/data/glom/data_128_subsample_8x/glom_split/test/glom
/repos/data/glom/data_128_subsample_8x/glom_split/val/normal
/repos/data/glom/data_128_subsample_8x/glom_split/val/glom


In [9]:
gen = gen_set(indir, outdir, df)
for nn, (ifn, ofn) in enumerate(gen):
    try:
        os.symlink(ifn, ofn)
    except FileExistsError as ee:
        print(ee)
        continue
    if nn % 1000 == 0:
        print(nn)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
